In [8]:
ligand_size = 0
step_number = []
number_of_finite_energy_samples = []
min_energies = []
mean_energies = []
std_energies = []
initial_center_of_masses = []
grid_volumes =[]
number_of_translations = []
fftlog = open('/home/jim/Desktop/FFT_notebook/4.receptor_grid/1AVZ_B:C/1avz_grid.log')
for line in fftlog:
    #print(line)
    if len(line.split()) > 2:    
        if line.split()[2] == 'atoms' :
            ligand_size = line.split()[6]
    if len(line.split()) == 5:
        if line.split()[3] == 'step':
            step_number.append(line.split()[4])
            #print(line)
    if len(line.split()) > 2:    
        if line.split()[2] == 'finite' :
            number_of_finite_energy_samples.append(line.split()[5])
    if len(line.split()) > 0:    
        if line.split()[0] == 'Min':
            min_energies.append(float(line.split()[2]))
            #print(line)
    if len(line.split()) > 0:    
        if line.split()[0] == 'Mean':
            mean_energies.append(float(line.split()[2]))
            #print(line)
    if len(line.split()) > 0:    
        if line.split()[0] == 'STD':
            std_energies.append(float(line.split()[2]))
            #print(line)
    if len(line.split()) > 0:    
        if line.split()[0] == 'Initial':
            x = line.split()[4]
            x = x.lstrip('[')
            y = line.split()[5]
            z = line.split()[6]
            z = z.rstrip(']')
            center_of_mass = [float(x), float(y), float(z)]
            initial_center_of_masses.append(center_of_mass)
            #.append(float(line.split()[2]))
            #print(line)
    if len(line.split()) > 0:    
        if line.split()[0] == 'Grid':
            grid_volumes.append(float(line.split()[2]))
            #print(line)
    if len(line.split()) > 2:    
        if line.split()[2] == 'translations' :
            number_of_translations.append(int(line.split()[3]))
            #print(line)
fft_steps = [list(x) for x in zip(step_number, number_of_finite_energy_samples, min_energies, mean_energies, initial_center_of_masses, grid_volumes, number_of_translations)]
steps_minimized = sorted(fft_steps,key=lambda x:x[2])

#print(fft_steps[0])
#print(steps_minimized[0])

fftlog.close()

In [ ]:
import netCDF4 as nc
import numpy as np

fft_path = '/home/jim/Desktop/FFT_notebook/5.fft_sampling/1AVZ_B:C'
fft_sample_nc = '%s/fft_sample.nc'%fft_path

rec_positions = nc.Dataset(fft_sample_nc, 'r').variables["rec_positions"][:]
lig_positions = nc.Dataset(fft_sample_nc, 'r').variables["lig_positions"][:]
lig_com = nc.Dataset(fft_sample_nc, 'r').variables["lig_com"][:]
volume = nc.Dataset(fft_sample_nc, 'r').variables["volume"][:]
nr_grid_points = nc.Dataset(fft_sample_nc, 'r').variables["nr_grid_points"][:]
exponential_sums = nc.Dataset(fft_sample_nc, 'r').variables["exponential_sums"][:]
log_of_divisors = nc.Dataset(fft_sample_nc, 'r').variables["log_of_divisors"][:]

mean_energy = nc.Dataset(fft_sample_nc, 'r').variables["mean_energy"][:]
min_energy = nc.Dataset(fft_sample_nc, 'r').variables["min_energy"][:]
resampled_energies = nc.Dataset(fft_sample_nc, 'r').variables["resampled_energies"][:]
resampled_trans_vectors = nc.Dataset(fft_sample_nc, 'r').variables["resampled_trans_vectors"][:]
print(min_energy[0])

#grid coordinates
origin = nc.Dataset(fft_sample_nc, 'r').variables["origin"][:]
x = nc.Dataset(fft_sample_nc, 'r').variables["x"][:]
y = nc.Dataset(fft_sample_nc, 'r').variables["y"][:]
z = nc.Dataset(fft_sample_nc, 'r').variables["z"][:]
d = nc.Dataset(fft_sample_nc, 'r').variables["d1"][:]
print('native pose:', str(resampled_trans_vectors[0][0]))
print('native pose:', resampled_energies[0][0])

print('test', np.where(resampled_energies ==-1412.9091438865273))
print('min energy pose:', resampled_trans_vectors[120][0])
print('min energy pose:', resampled_energies[120][0])

for i, vec in enumerate(lig_com):
    adj_vec = []
    for x in vec:
        adj_vec.append(x/0.5)
    print('lig_com:%s'%i, vec, adj_vec)

for i, vec in enumerate(resampled_trans_vectors[0]):
    adj_vec = []
    for x in vec:
        adj_vec.append(x*0.5)
    print('vec%s:'%i, str(adj_vec), str(vec))


In [3]:
import netCDF4 as nc
import numpy as np

grid_path = '/home/jim/Desktop/FFT_notebook/4.receptor_grid/1AVZ_B:C'
grid_nc = '%s/grid.nc'%grid_path


#parsing some of the netcdf variables from grid.nc
grid_variables = nc.Dataset(grid_nc, 'r').variables
counts = nc.Dataset(grid_nc, 'r').variables["counts"][:]
x = nc.Dataset(grid_nc, 'r').variables["x"][:]
y = nc.Dataset(grid_nc, 'r').variables["y"][:]
z = nc.Dataset(grid_nc, 'r').variables["z"][:]
electrostatic = nc.Dataset(grid_nc, 'r').variables["electrostatic"][:]
lja = nc.Dataset(grid_nc, 'r').variables["LJa"][:]
ljr = nc.Dataset(grid_nc, 'r').variables["LJr"][:]
occupancy = nc.Dataset(grid_nc, 'r').variables["occupancy"][:]
trans_crd = nc.Dataset(grid_nc, 'r').variables["trans_crd"][:]

#DELETE ME: print statements for inspecting the contents of grid.nc
#print(len(occupancy[200][:]))
#print(grid_variables)
kttc = []
for key in grid_variables:
    kttc.append(key)
#    print(grid_variables[key])
print(trans_crd)
# print(counts)
#electrostatic grid is accessed by grid[x][y][z], to be confirmed. Alternative is grid[z][y][x]
#below code accesses electrostatic grid values for 3 points on the same axis
print(electrostatic[0][0][0],electrostatic[0][0][1],electrostatic[0][0][2])
#identifies occupied positions on the grid 
occupied_list = [i for i in range(len(occupancy[50][50])) if occupancy[50][50][i] == 1] 
          
# printing resultant list  
print ("New indices list : " + str(occupied_list)) 

#for i in res_list:
#    print(electrostatic[200][200][i])
    
#print(trans_crd)
#print(trans_crd[:])

# def multiply_grids(electrostatic_grid, lja_grid, ljr_grid, occupancy_grid):
#     grid_dot = np.empty((electrostatic_grid.shape[0], electrostatic_grid.shape[1], electrostatic_grid.shape[2]))
#     for i in range(electrostatic_grid.shape[0]):
#         for j in range(electrostatic_grid.shape[1]):
#             for k in range(electrostatic_grid.shape[2]):
#                 grid_dot[i][j][k] = np.dot(electrostatic_grid[i][j][k], lja_grid[i][j][k])
#                 grid_dot[i][j][k] = np.dot(grid_dot[i][j][k], ljr_grid[i][j][k])
#     return grid_dot

# def get_meaningful_energies(energies):
#     for i in range(energies.shape[0]):
#         for j in range(energies.shape[1]):
#             for k in range(energies.shape[2]):
#                 if energies[i][j][k] > 0.5:
#                     print(energies[i][j][k])


# result = multiply_grids(electrostatic, lja, ljr, occupancy)
# get_meaningful_energies(result)
# #print(result)
# for i in trans_crd:
#     if i[0] > 140:
#         print(i)
    
# native_trans = [i for i in range(len(trans_crd[:])) if trans_crd[i][0] == 140] 
# print(len(trans_crd))
# for point in occupied_list:
#     print(trans_crd[point])
# get the center of mass of the crystallographic pose, then, get the center of mass for every
# translation of the native rotation and measure the distance between the two points. Select
# the translation with the nearest center of mass as the "native/crystallographic pose"



[[73.94838745 38.87543675 35.58284755]
 [74.65255345 39.59851505 35.67783355]
 [73.10307885 39.33137175 35.26389115]
 ...
 [56.84128645 81.68257475 72.28708265]
 [55.98825335 80.79374075 72.09547035]
 [56.91710595 82.70364525 71.56762125]]
4.42511968959854 4.436090157643713 4.44698288906925
New indices list : []


In [ ]:
print(result)